# Examining Data of Gun Related Incidents
I've taken the compiled and cleaned data provided by https://github.com/fivethirtyeight/guns-data and tried analyze it.

first, I begin by reading the file, and converting it into a list.

In [39]:
import csv
import datetime
from __future__ import division

year_counts = {}
date_counts = {}
sex_counts = {}
race_counts = {}
rate = 100000

ofile = csv.reader(open("guns-data.csv", 'r'))
data = list(ofile)


I realize that the data contains headers, so I seperate the top row.
Once that's done, I take the the data from all the coloumns I want to analyze and assign it to their respective variables.

In [40]:
headers = data[0]
data = data[1:]

years = [row[1] for row in data]
dates = [datetime.datetime(year=int(row[1]), month=int(row[2]), day=1) for row in data]
race = [row[7] for row in data]
sex = [row[5] for row in data]

My intention now is to tally all the data in related to the coloumn info. When working with it earlier, I realized that it might be a repetitive task, so I created a generaly function which took in the coloumn data as a parameter and assigned the tallies to a dictionary.The function then returns the dictionary which could be assigned to a properly named variable outside the function. 

In [41]:
def tally_coloumn(coloumn_list):
    tally_dict = {}
    for row in coloumn_list:
        if row not in tally_dict:
            tally_dict[row] = 1
        else:
            tally_dict[row] += 1
    return tally_dict

year_counts = tally_coloumn(years)
date_counts = tally_coloumn(dates)
race_counts = tally_coloumn(race)
sex_counts = tally_coloumn(sex)

In [51]:
census_ofile = csv.reader(open("census.csv", 'r'))
census_data = list(census_ofile)[1:]
print(census_data)
mappings = {
    'Asian/Pacific Islander': float(census_data[0][14]+ census_data[0][15]),
    'Black': float(census_data[0][12]),
    'Native American/Native Alaskan': float(census_data[0][13]),
    'Hispanic': float(census_data[0][11]),
    'White': float(census_data[0][10])
}
print(mappings['White'])

[['cen42010', 'April 1, 2010 Census', 'totsex', 'Both Sexes', 'tothisp', 'Total', '0100000US', '', 'United States', '308745538', '197318956', '44618105', '40250635', '3739506', '15159516', '674625', '6984195']]
197318956.0


In [53]:
white_gun_death_rate_perhunK = race_counts['White'] / mappings['White'] * rate
print(white_gun_death_rate_perhunK)

33.5684930342
